In [146]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [147]:
data_path = 'data/'

train_csv_file_path = data_path + 'train_ship_segmentations_v2.csv'
train_image_path = data_path + 'train_v2/'

In [148]:
def load_csv(path):
    return tf.data.experimental.make_csv_dataset(
        path,
        batch_size=32, # required
        column_names=['ImageId', 'EncodedPixels'],
    )

train_csv = load_csv(train_csv_file_path)

# for batch in train_csv.take(1):
#     print(batch)

In [156]:
def process_img(file):
    img = tf.io.read_file(train_image_path + file)
    img = tf.io.decode_jpeg(img, channels=3)
    return img

def process_label(label):
    return label

def process_batch(csv_item):
    X = process_img(csv_item['ImageId'])
    y = process_label(csv_item['EncodedPixels'])
    return X, y

train_ds = train_csv.unbatch().map(process_batch)

# for item, label in train_ds.take(1):
#     plt.imshow(item)
#     print(label)